In [1]:
#importar as bibliotecas
import pandas as pd
from imap_tools import MailBox, AND
import yagmail
import pathlib
import yaml
import logging

#importar base de dados
emails = pd.read_excel(r"Bases de Dados\Emails.xlsx")
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

In [2]:
#incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')

In [3]:
dicionario_lojas={}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

In [4]:
dia_indicador = vendas['Data'].max()

In [13]:
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo

    dicionario_lojas[loja].to_excel(local_arquivo)

In [6]:
# definição de metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtd_produtos_dia = 4
meta_qtd_produtos_ano = 120
meta_ticket_medio_dia = 500
meta_ticket_medio_ano = 500

In [7]:


def load_credentials(filepath):
    try:
        with open(filepath, 'r') as file:
            credentials = yaml.safe_load(file)
            user = credentials['user']
            password = credentials['password']
            return user, password
    except Exception as e:
        logging.error("Failed to load credentials: {}".format(e))
        raise

In [24]:
for loja in dicionario_lojas:
  vendas_loja = dicionario_lojas[loja]
  vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

  # faturamento
  faturamento_ano = vendas_loja['Valor Final'].sum()
  faturamento_dia = vendas_loja_dia['Valor Final'].sum()

  # diversidade de produtos
  qtde_produtos_ano = len(vendas_loja['Produto'].unique())
  qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

  # tiket medio
  vendas_loja = vendas_loja.drop(columns=vendas_loja.select_dtypes(include='datetime64').columns)
  valor_venda = vendas_loja.groupby('Código Venda').sum()
  ticket_medio_ano = valor_venda['Valor Final'].mean()

  valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)
  ticket_medio_dia = valor_venda_dia['Valor Final'].mean()


  #enviar email
  credentials = load_credentials('docs/credentials.yaml')
  meu_email = yagmail.SMTP(*credentials)
  nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
  # email = 'nevertonali01@gmail.com'
  email = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
  titulo = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

  if faturamento_dia >= meta_faturamento_dia:
    cor_fat_dia = 'green'
  else:
    cor_fat_dia = 'red'

  if faturamento_ano >= meta_faturamento_ano:
    cor_fat_ano = 'green'
  else:
    cor_fat_ano = 'red'
    
  if qtde_produtos_dia >= meta_qtd_produtos_dia:
    cor_qtde_dia = 'green'
  else:
    cor_qtde_dia = 'red'

  if qtde_produtos_ano >= meta_qtd_produtos_ano:
    cor_qtde_ano = 'green'
  else:
    cor_qtde_ano = 'red'

  if ticket_medio_dia >= meta_ticket_medio_dia:
    cor_ticket_dia = 'green'
  else:
    cor_ticket_dia = 'red'

  if ticket_medio_ano >= meta_qtd_produtos_ano:
    cor_ticket_ano = 'green'
  else:
    cor_ticket_ano = 'red'


  corpo = f'''
  <p>Bom dia, {nome}</p>
  <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja} foi:<?strong></p>
  <table>
    <tr>
      <th>Indicador</th>
      <th>Valor Dia</th>
      <th>Meta Dia</th>
      <th>Cenário Dia</th>
    </tr>
    <tr>
      <td>Faturamento</td>
      <td style="text-align: center">R${faturamento_dia:.2f}</td>
      <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
      <td style="text-align: center"><font color="{cor_fat_dia}">X</font></td>
    </tr>
    <tr>
      <td>Diversidade de Produtos</td>
      <td style="text-align: center">{qtde_produtos_dia}</td>
      <td style="text-align: center">{meta_qtd_produtos_dia}</td>
      <td style="text-align: center"><font color="{cor_qtde_dia}">X</font></td>
    </tr>
    <tr>
      <td>Ticket Médio</td>
      <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
      <td style="text-align: center">R${meta_ticket_medio_dia:.2f}</td>
      <td style="text-align: center"><font color="{cor_ticket_dia}">X</font></td>
    </tr>
  </table>
  <table>
    <tr>
      <th>Indicador</th>
      <th>Valor Ano</th>
      <th>Meta Ano</th>
      <th>Cenário Ano</th>
    </tr>
    <tr>
      <td>Faturamento</td>
      <td style="text-align: center">R${faturamento_ano:.2f}</td>
      <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
      <td style="text-align: center"><font color="{cor_fat_ano}">X</font></td>
    </tr>
    <tr>
      <td>Diversidade de Produtos</td>
      <td style="text-align: center">{qtde_produtos_ano}</td>
      <td style="text-align: center">{meta_qtd_produtos_ano}</td>
      <td style="text-align: center"><font color="{cor_qtde_ano}">X</font></td>
    </tr>
    <tr>
      <td>Ticket Médio</td>
      <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
      <td style="text-align: center">R${meta_ticket_medio_ano:.2f}</td>
      <td style="text-align: center"><font color="{cor_ticket_ano}">X</font></td>
    </tr>
  </table>
  <p>Segue em anexo a planilha com todos od dados para mais detalhes.</p>
  <p>Qualquer dúvida estou à disposição</p>
  <p>Att., Neverton</p>
  '''

  #anexos 
  anexos = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
  meu_email.send(to=email, subject=titulo, contents=corpo, attachments=anexos)
  print(f'Email da loja {loja} enviado!')

Email da loja Iguatemi Esplanada enviado!
Email da loja Shopping Midway Mall enviado!
Email da loja Norte Shopping enviado!
Email da loja Shopping Iguatemi Fortaleza enviado!
Email da loja Shopping União de Osasco enviado!
Email da loja Shopping Center Interlagos enviado!
Email da loja Rio Mar Recife enviado!
Email da loja Salvador Shopping enviado!
Email da loja Rio Mar Shopping Fortaleza enviado!
Email da loja Shopping Center Leste Aricanduva enviado!
Email da loja Ribeirão Shopping enviado!
Email da loja Shopping Morumbi enviado!
Email da loja Parque Dom Pedro Shopping enviado!
Email da loja Bourbon Shopping SP enviado!
Email da loja Palladium Shopping Curitiba enviado!
Email da loja Passei das Águas Shopping enviado!
Email da loja Center Shopping Uberlândia enviado!
Email da loja Shopping Recife enviado!
Email da loja Shopping Vila Velha enviado!
Email da loja Shopping SP Market enviado!
Email da loja Shopping Eldorado enviado!
Email da loja Shopping Ibirapuera enviado!
Email da lo